In [1]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.cmi_inference_server

In [2]:
# Importing needed libraries
import numpy as np
import pandas as pd
import sklearn as sk

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in the gateway will run in a different container with direct access to the hidden test set and hand off the one sequence at a time.

Your code will always have access to the published copies of the files.

In [3]:
'''
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    # Replace this function with your inference code.
    # You can return either a Pandas or Polars dataframe, though Polars is recommended.
    # Each prediction (except the very first) must be returned within 30 minutes of the batch features being provided.
    return 'Text on phone'
'''

"\ndef predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:\n    # Replace this function with your inference code.\n    # You can return either a Pandas or Polars dataframe, though Polars is recommended.\n    # Each prediction (except the very first) must be returned within 30 minutes of the batch features being provided.\n    return 'Text on phone'\n"

In [4]:
#Checking how test data looks
file_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv'

# Reading the CSV file
df_test_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
df_test_data.head(100)

,row_id,sequence_id,sequence_counter,subject,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000001_000000,SEQ_000001,0,SUBJ_055840,9.039062,5.261719,0.800781,0.367188,-0.397400,-0.629028,...,97.0,87.0,206.0,-1.0,195.0,-1.0,-1.0,-1.0,111.0,-1.0
1,SEQ_000001_000001,SEQ_000001,1,SUBJ_055840,9.421875,3.460938,-1.113281,0.353882,-0.507141,-0.652710,...,175.0,158.0,-1.0,-1.0,-1.0,-1.0,-1.0,211.0,187.0,178.0
2,SEQ_000001_000002,SEQ_000001,2,SUBJ_055840,10.160156,2.082031,-3.871094,0.384094,-0.532104,-0.639648,...,-1.0,160.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,197.0,177.0
3,SEQ_000001_000003,SEQ_000001,3,SUBJ_055840,9.773438,1.355469,-4.371094,0.387756,-0.531982,-0.634033,...,-1.0,160.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,197.0,183.0
4,SEQ_000001_000004,SEQ_000001,4,SUBJ_055840,9.195312,1.011719,-3.222656,0.382751,-0.534180,-0.638367,...,-1.0,163.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,200.0,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SEQ_000011_000039,SEQ_000011,39,SUBJ_016452,-4.523438,8.648438,-2.605469,0.370117,0.188660,0.786682,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
96,SEQ_000011_000040,SEQ_000011,40,SUBJ_016452,-3.949219,8.652344,-3.292969,0.367615,0.185852,0.788025,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
97,SEQ_000011_000041,SEQ_000011,41,SUBJ_016452,-4.062500,8.691406,-3.253906,0.367676,0.183411,0.785461,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
98,SEQ_000011_000042,SEQ_000011,42,SUBJ_016452,-3.527344,8.535156,-3.140625,0.368286,0.189819,0.788391,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [5]:
# Here first row from test data is taken and prediction returned, 
# the second time the funtion is called the second row of test data is taken and prediction returned and so on.

# Global counter to keep track of which test row to use
current_test_index = 0

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    global current_test_index
    
    # Loading the CSV file
    file_path = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv'
    
    # Reading the CSV file
    df_original_data = pd.read_csv(file_path)

    # Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
    #df_original_data.head()

    # Checking number of null values in each column
    #df_original_data.isnull().sum()

    # Defining feature and target
    A = df_original_data.iloc[:, 9:].values
    b = df_original_data.iloc[:, 8].values

    #print(A)
    #print(b)

    # Now the feature column has some null values - imputing them with a constant value Zero    
    # Replace NaNs with 0
    A = df_original_data.iloc[:, 9:].values
    A = np.nan_to_num(A, nan=0)
    #print(A[:5])  # Display first 5 rows

    # Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    A_train_s = sc.fit_transform(A)

    # Instead of test train split, using whole of the data to train our model.

    # Fitting KNN to the training set

    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(A_train_s, b)

    # Importing the testing data 

    # Convert Polars DataFrame to Pandas 
    df_original_test_data = sequence.to_pandas()

    # Display the first few rows of the dataframe, we will be doing this as checking step every step along the way
    #df_original_test_data.head()

    print(current_test_index)
    
    # Only taking needed columns from test data, and the needed row
    original_test_data = df_original_test_data.iloc[current_test_index, 4:].values

    # To show current row_id, sequence_id, sequence_counter data being tested out.
    row_id = df_original_test_data['row_id'].iloc[current_test_index]
    sequence_id = df_original_test_data['sequence_id'].iloc[current_test_index]
    sequence_counter = df_original_test_data['sequence_counter'].iloc[current_test_index]
    #print(row_id, ",", sequence_id, ",", sequence_counter) 
    
    current_test_index += 1 # To update the row value, when the funtion is called next time
    
    # Reshape is done coz above code gives a 1D shape while the input for next is a 2D shape, just a small tweak.
    original_test_data = original_test_data.reshape(1, -1)
    #print(original_test_data)

    # Predicting the test set - results
    y_pred = classifier.predict(original_test_data)
    #print(y_pred)

    y_pred_string = str(y_pred[0]) # Converting the return value to a string.
    #print(y_pred_string)
    #print(sequence_id, y_pred_string)
    print(row_id, ",", sequence_id, ",", sequence_counter, ",", y_pred_string)
    
    #print('Checkpoint 1 ~ Akash')
    return y_pred_string

    # ~ End of code.

In [6]:
'''
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )
'''

"\ninference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)\n\nif os.getenv('KAGGLE_IS_COMPETITION_RERUN'):\n    inference_server.serve()\nelse:\n    inference_server.run_local_gateway(\n        data_paths=(\n            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',\n            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',\n        )\n    )\n"

In [7]:
# Loading the CSV file
file_path_test = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv'
    
# Reading the CSV file
df_original_test_data = pd.read_csv(file_path_test)

# Get number of rows in test data
num_rows = df_original_test_data.shape[0]
print('Number of row in test data = ', num_rows)

print('row_id, sequence_id, sequence_counter, gesture prediction')

# Call predict() for each row
for i in range(num_rows):    
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        print('Checkpoint 2 ~ Akash')
        inference_server.serve()
    else:
        #print('Checkpoint 2.5 ~ Akash')
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )
    #print('Checkpoint 3 ~ Akash')

Number of row in test data =  107
row_id, sequence_id, sequence_counter, gesture prediction
0
SEQ_000011_000000 , SEQ_000011 , 0 , Pull air toward your face
1
SEQ_000001_000001 , SEQ_000001 , 1 , Wave hello
2
SEQ_000001_000002 , SEQ_000001 , 2 , Wave hello
3
SEQ_000011_000003 , SEQ_000011 , 3 , Text on phone
4
SEQ_000001_000004 , SEQ_000001 , 4 , Wave hello
5
SEQ_000011_000005 , SEQ_000011 , 5 , Text on phone
6
SEQ_000001_000006 , SEQ_000001 , 6 , Wave hello
7
SEQ_000011_000007 , SEQ_000011 , 7 , Text on phone
8
SEQ_000001_000008 , SEQ_000001 , 8 , Wave hello
9
SEQ_000011_000009 , SEQ_000011 , 9 , Wave hello
10
SEQ_000011_000010 , SEQ_000011 , 10 , Pull air toward your face
11
SEQ_000001_000011 , SEQ_000001 , 11 , Wave hello
12
SEQ_000001_000012 , SEQ_000001 , 12 , Wave hello
13
SEQ_000011_000013 , SEQ_000011 , 13 , Cheek - pinch skin
14
SEQ_000001_000014 , SEQ_000001 , 14 , Cheek - pinch skin
15
SEQ_000011_000015 , SEQ_000011 , 15 , Wave hello
16
SEQ_000001_000016 , SEQ_000001 , 16 , 

/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/templates.py:136: RuntimeWarning: 930 seconds elapsed before server startup.
                This exceeds the startup time limit of 900 seconds that the gateway will enforce
                during the rerun on the hidden test set. Start the server before performing any time consuming steps.
  warnings.warn(


26
SEQ_000001_000026 , SEQ_000001 , 26 , Cheek - pinch skin
27
SEQ_000011_000027 , SEQ_000011 , 27 , Forehead - pull hairline
28
SEQ_000001_000028 , SEQ_000001 , 28 , Cheek - pinch skin
29
SEQ_000011_000029 , SEQ_000011 , 29 , Forehead - pull hairline
30
SEQ_000011_000030 , SEQ_000011 , 30 , Write name in air
31
SEQ_000001_000031 , SEQ_000001 , 31 , Cheek - pinch skin
32
SEQ_000011_000032 , SEQ_000011 , 32 , Forehead - pull hairline
33
SEQ_000001_000033 , SEQ_000001 , 33 , Cheek - pinch skin
34
SEQ_000001_000034 , SEQ_000001 , 34 , Cheek - pinch skin
35
SEQ_000011_000035 , SEQ_000011 , 35 , Forehead - pull hairline
36
SEQ_000001_000036 , SEQ_000001 , 36 , Cheek - pinch skin
37
SEQ_000011_000037 , SEQ_000011 , 37 , Forehead - pull hairline
38
SEQ_000011_000038 , SEQ_000011 , 38 , Forehead - pull hairline
39
SEQ_000001_000039 , SEQ_000001 , 39 , Cheek - pinch skin
40
SEQ_000011_000040 , SEQ_000011 , 40 , Forehead - pull hairline
41
SEQ_000001_000041 , SEQ_000001 , 41 , Cheek - pinch skin

GatewayRuntimeError: (<GatewayRuntimeErrorType.SERVER_RAISED_EXCEPTION: 3>, 'single positional indexer is out-of-bounds')